<a href="https://colab.research.google.com/github/xBrymer/COVID19-AI-CT-Scan-Detection/blob/master/dataset_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
import os
import shutil
import cv2
import random
from tqdm.notebook import tqdm
from zipfile import ZipFile
from os.path import join


# Importing the first dataset and transcoding it

In [ ]:
raw_dir = '/content/raw'

ds1_dir = '/content/raw/ds1'
ds2_dir = '/content/raw/ds2'

In [ ]:
try:
  os.mkdir(raw_dir)
  os.mkdir(ds1_dir)
  os.mkdir(ds2_dir)
except:
  pass

In [ ]:
os.chdir(raw_dir)

Downloading the first dataset

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1N0jmtIJkzRrPJHpXpk8mlM5JPirL6_Gb' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1N0jmtIJkzRrPJHpXpk8mlM5JPirL6_Gb" -O covid19-ctset.zip && rm -rf /tmp/cookies.txt

In [ ]:
file_path = '/content/raw/covid19-ctset.zip'

f = ZipFile(file_path, 'r')
f.extractall(ds1_dir)

In [ ]:
ds1_converted_dir = '/content/converted'
os.mkdir(ds1_converted_dir)

In [ ]:
import PIL.Image as pil_image
import io
import numpy as np

for image in tqdm(os.listdir(ds1_dir)):
  path= join(ds1_dir, image)
  save_path= join(ds1_converted_dir, image)
  with open(path, 'rb') as f:
      tif = pil_image.open(io.BytesIO(f.read()))
  array=np.array(tif)
  max_val=np.amax(array)
  normalized=(array/max_val)
  im = pil_image.fromarray(normalized)
  im.save(save_path) 


In [ ]:
ds1_main = '/content/ds1'
ds1_covid = '/content/ds1/covid'
ds1_non_covid = '/content/ds1/non-covid'

os.mkdir(ds1_main)
os.mkdir(ds1_covid)
os.mkdir(ds1_non_covid)


In [20]:
def save_as_jpg(source):
  count = 0
  current_dir = os.getcwd()

  for image in tqdm(os.listdir(source)):
    path = join(source, image)
    count +=1
    img = cv2.imread(path, -1)
    
  
    if 'covid' in image:
      name = f'covid-{count}.jpg'
      destination = ds1_covid

    elif 'normal' in image:
      name = f'normal-{count}.jpg'
      destination = ds1_non_covid

    os.chdir(destination)
    cv2.imwrite(name, img, [cv2.IMWRITE_JPEG_QUALITY, 100])
  
  os.chdir(current_dir)
  return



    

In [18]:
print(os.getcwd())

/content/raw


In [19]:
for image in os.listdir(raw_dir):
  if image.endswith('.jpg'):
    os.remove(join(raw_dir, image))

In [21]:
save_as_jpg('/content/converted')

# Importing the second dataset

In [22]:
# connect to google drive

In [23]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/My Drive/External Datasets/Kaggle"

In [24]:
%cd /content/raw/ds2

/content/raw/ds2


In [25]:
!kaggle datasets download -d azaemon/preprocessed-ct-scans-for-covid19

100% 3.57G/3.59G [00:44<00:00, 93.1MB/s]
100% 3.59G/3.59G [00:44<00:00, 85.8MB/s]


In [26]:
path = '/content/raw/ds2/preprocessed-ct-scans-for-covid19.zip'

f = ZipFile(path, 'r')
f.extractall('/content/raw/ds2')

# Splitting and balancing the data

In [83]:
main_dir = '/content/main'

train_dir = join(main_dir, 'train')

train_c_dir = join(train_dir, 'covid')
train_nc_dir = join(train_dir, 'non-covid')
train_ni_dir = join(train_dir, 'non-informative')

test_dir = join(main_dir, 'test')

test_c_dir = join(test_dir, 'covid')
test_nc_dir = join(test_dir, 'non-covid')
test_ni_dir = join(test_dir, 'non-informative')

val_dir = join(main_dir, 'validation')

val_c_dir = join(val_dir, 'covid')
val_nc_dir = join(val_dir, 'non-covid')
val_ni_dir = join(val_dir, 'non-informative')

dirs = [
  train_dir,
 train_c_dir,
  train_nc_dir,
  train_ni_dir,

  test_dir,
  test_c_dir,
  test_nc_dir,
  test_ni_dir,

  val_dir,
  val_c_dir,
  val_nc_dir,
  val_ni_dir,
]

try:
  for dir in dirs:
    os.mkdir(dir)
    print(f'Made directory: {dir}')
except:
  pass

Made directory: /content/main/train
Made directory: /content/main/train/covid
Made directory: /content/main/train/non-covid
Made directory: /content/main/train/non-informative
Made directory: /content/main/test
Made directory: /content/main/test/covid
Made directory: /content/main/test/non-covid
Made directory: /content/main/test/non-informative
Made directory: /content/main/validation
Made directory: /content/main/validation/covid
Made directory: /content/main/validation/non-covid
Made directory: /content/main/validation/non-informative


In [72]:
print(os.listdir(main_dir))

['train', 'validation', 'test']


In [82]:
for folder in os.listdir(main_dir):
  shutil.rmtree(join(main_dir, folder))

In [65]:
def copy_images(source, destination):

  for image in tqdm(os.listdir(source)):
    source_path = join(source, image)
    destination_path = join(destination, image)
    shutil.copy(source_path, destination_path)
  
  return

In [84]:
copy_images('/content/raw/ds2/Original CT Scans/NiCT', train_ni_dir)
copy_images('/content/raw/ds2/Original CT Scans/nCT', train_nc_dir)
copy_images('/content/raw/ds2/Original CT Scans/pCT', train_c_dir)

In [74]:
total_c = len(os.listdir(train_c_dir)) + len(os.listdir(ds1_covid))
total_nc = len(os.listdir(train_nc_dir)) + len(os.listdir(ds1_non_covid))
total_ni = len(os.listdir(train_ni_dir))

print(f'total covid images = {total_c}')
print(f'total non-covid images = {total_nc}')
print(f'total non-informative images = {total_ni}')

print()

print(f'train covid images = {round(total_c*(.70))}')
print(f'train non-covid images = {round(total_nc*(.70))}')
print(f'train non-informative images = {round(total_ni*(.70))}')

print()

print(f'val covid images = {round(total_c*(.20))}')
print(f'val non-covid images = {round(total_nc*(.20))}')
print(f'val non-informative images = {round(total_ni*(.20))}')

print()

print(f'test covid images = {round(total_c*(.10))}')
print(f'test non-covid images = {round(total_nc*(.10))}')
print(f'test non-informative images = {round(total_ni*(.10))}')

total covid images = 6283
total non-covid images = 19755
total non-informative images = 5705

train covid images = 4398
train non-covid images = 13828
train non-informative images = 3993

val covid images = 1257
val non-covid images = 3951
val non-informative images = 1141

test covid images = 628
test non-covid images = 1976
test non-informative images = 570


In [85]:
def distribute(total, source, destination1, destination2):
  for i in tqdm(range(round(total*(.30)))):
    image = random.choice(os.listdir(source))

    if i % 3 == 0:
      destination = destination1
    elif i % 3 == 1:
      destination = destination1
    elif i % 3 == 2:
      destination = destination2
  
    img_path = join(source, image)
    destination_path = join(destination, image)

    shutil.move(img_path, destination_path)
  
  return


In [86]:
distribute(6283, train_c_dir, val_c_dir, test_c_dir)
distribute(19755, train_nc_dir, val_nc_dir, test_nc_dir)
distribute(5705, train_ni_dir, val_ni_dir, test_ni_dir)

In [87]:

copy_images('/content/ds1/non-covid', train_nc_dir)
copy_images('/content/ds1/covid', train_c_dir)

In [90]:
print(f'train covid images = {len(os.listdir(train_c_dir))}')
print(f'train non-covid images = {len(os.listdir(train_nc_dir))}')
print(f'train non-informative images = {len(os.listdir(train_ni_dir))}')

print()

print(f'val covid images = {len(os.listdir(val_c_dir))}')
print(f'val non-covid images = {len(os.listdir(val_nc_dir))}')
print(f'val non-informative images = {len(os.listdir(val_ni_dir))}')

print()

print(f'test covid images = {len(os.listdir(test_c_dir))}')
print(f'test non-covid images = {len(os.listdir(test_nc_dir))}')
print(f'test non-informative images = {len(os.listdir(test_ni_dir))}')

train covid images = 4398
train non-covid images = 13829
train non-informative images = 3993

val covid images = 1257
val non-covid images = 3951
val non-informative images = 1142

test covid images = 628
test non-covid images = 1975
test non-informative images = 570


In [91]:
difference = len(os.listdir(train_nc_dir)) - len(os.listdir(train_c_dir)) 
print(difference)

9431


In [92]:
for i in tqdm(range(9431)):
  image = random.choice(os.listdir(train_nc_dir))
  os.remove(join(train_nc_dir, image))

In [93]:
print(f'train covid images = {len(os.listdir(train_c_dir))}')
print(f'train non-covid images = {len(os.listdir(train_nc_dir))}')
print(f'train non-informative images = {len(os.listdir(train_ni_dir))}')

print()

print(f'val covid images = {len(os.listdir(val_c_dir))}')
print(f'val non-covid images = {len(os.listdir(val_nc_dir))}')
print(f'val non-informative images = {len(os.listdir(val_ni_dir))}')

print()

print(f'test covid images = {len(os.listdir(test_c_dir))}')
print(f'test non-covid images = {len(os.listdir(test_nc_dir))}')
print(f'test non-informative images = {len(os.listdir(test_ni_dir))}')

train covid images = 4398
train non-covid images = 4398
train non-informative images = 3993

val covid images = 1257
val non-covid images = 3951
val non-informative images = 1142

test covid images = 628
test non-covid images = 1975
test non-informative images = 570


In [94]:
zip_path = shutil.make_archive('main_dataset','zip', root_dir = '/content', base_dir = 'main', verbose = 2)

In [95]:
print(zip_path)

/content/raw/ds2/main_dataset.zip


In [96]:
shutil.copy(zip_path, '/content/drive/My Drive/External Datasets/COVID19-Project/Datasets')

'/content/drive/My Drive/External Datasets/COVID19-Project/Datasets/main_dataset.zip'